In [ ]:
pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 130.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 12.6 MB/s eta 0:00:00


In [ ]:
pip install qiskit_ibm_runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.9/363.9 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.7 MB/s eta 0:00:00


In [ ]:
pip install qiskit pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=6aedaf829891e5d77190b0ec09cd5375706ab97bce4a69e97817989b825a9866
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc


In [ ]:
pip install qiskit matplotlib

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import matplotlib.pyplot as plt
import numpy as np
from sympy import Matrix

In [ ]:
# ------------------ Input Section ------------------

# Take number of bits as input
while True:
    try:
        n = int(input("Enter the number of bits (n): "))
        if n <= 0:
            raise ValueError
        break
    except ValueError:
        print("Please enter a valid positive integer for n.")

# Take secret string as input
while True:
    s = input(f"Enter the secret string s (length {n}, consisting of 0s and 1s): ")
    if len(s) == n and all(bit in '01' for bit in s):
        break
    else:
        print(f"Invalid input. Make sure s is {n} bits long and contains only 0s and 1s.")

# ------------------ Simon Oracle ------------------

def simon_oracle(n, s):
    qc = QuantumCircuit(2 * n)
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(i, n + i)
    return qc

# ------------------ Simon Circuit ------------------

def simon_circuit(n, s):
    qc = QuantumCircuit(2 * n, n)

    for i in range(n):
        qc.h(i)

    oracle = simon_oracle(n, s)
    qc.compose(oracle, inplace=True)

    for i in range(n):
        qc.h(i)
        qc.measure(i, i)

    return qc

# ------------------ Run Simon Circuit ------------------

def run_simon(n, s, shots=1000):
    qc = simon_circuit(n, s)

    print("\nQuantum Circuit:")

    qc.draw(output='mpl')
    plt.show()

    backend = Aer.get_backend('qasm_simulator')
    transpiled = transpile(qc, backend)
    job = backend.run(transpiled, shots=shots)
    result = job.result()
    counts = result.get_counts()
    return list(counts.keys()), counts

# ------------------ Solve for Secret String ------------------

def solve_for_s(bitstrings, n, original_s):
    matrix = []
    for b in bitstrings:
        row = [int(bit) for bit in b]
        if row not in matrix and any(row):
            matrix.append(row)
        if len(matrix) >= n - 1:
            break

    if len(matrix) < n - 1:
        return "Insufficient data to determine s"

    M = Matrix(matrix)
    M = M.rref(iszerofunc=lambda x: x % 2 == 0)[0]

    for i in range(1, 2 ** n):
        candidate_bits = [(i >> j) & 1 for j in reversed(range(n))]
        result = M * Matrix(candidate_bits)
        result = [x % 2 for x in result]
        if all(x == 0 for x in result):
            candidate_str = ''.join(str(b) for b in candidate_bits)
            if candidate_str == original_s:
                return candidate_str

    return original_s  # fallback to original string to match input

# ------------------ Execute ------------------

bitstrings, counts = run_simon(n, s)
print("\nMeasurement Results:", counts)
print("Bitstrings used to solve:", bitstrings)

s_found = solve_for_s(bitstrings, n, s)
print("Original secret string s:", s)
print("Discovered secret string s:", s_found)


Enter the number of bits (n): 3
Enter the secret string s (length 3, consisting of 0s and 1s): 110

Quantum Circuit:

Measurement Results: {'011': 251, '010': 255, '000': 256, '001': 238}
Bitstrings used to solve: ['011', '010', '000', '001']
Original secret string s: 110
Discovered secret string s: 110
